# 迁移学习

用别人训练好的权重参数当咱们模型的初始化

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

In [23]:
base_dir = 'D:/jwj/tensorflow/cats_and_dogs'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'valid')

train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_cats_dir = os.path.join(test_dir,'cats')
validation_dogs_dir = os.path.join(test_dir,'dogs')

## tensorflow.keras.applications

In [3]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import ResNet101

In [4]:
# include_top：要不要加全连接
pre_train_model = ResNet101(input_shape=(75,75,3),include_top=False,weights='imagenet')

In [5]:
for layer in pre_train_model.layers:
    layer.trainable = False

## callback的作用

在训练过程中可以设置一些自定义项，比如提前停止，改变学习率

In [6]:
# 在fit_generator中加入
# tf.keras.callback.EarlyStopping(patience=2,monitor='val_loss')

# tf.keras.callback.LearningRateScheduler

# 自定义
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if (logs.get('acc')>0.95):
            print("Reaching 95% accuracy so stop training !")
            self.model.stop_training=True

In [7]:
from tensorflow.keras.optimizers import Adam

#为全连接层做准备
x = layers.Flatten()(pre_train_model.output)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.2)(x)
# 输出层
x = layers.Dense(1,activation='sigmoid')(x)
# 构建模型序列：知道头模块和尾模块
model = Model(pre_train_model.input,x)

model.compile(optimizer=Adam(lr=0.001),
             loss='binary_crossentropy',
             metrics=['acc'])

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(75,75),
                                                    batch_size=20,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(75,75),
                                                    batch_size=20,
                                                    class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [25]:
callback = myCallback()
history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=100,
                             validation_data = validation_generator,
                             validation_steps = 50,
                             verbose=2,
                             callbacks=[callback])

OSError: [Errno 22] Invalid argument: 'D:/jwj/tensorflow/cats_and_dogs\\train\\dogs\\dog.11487.jpg'

In [ ]:
# 选择需要的层
# last_layer = pre_train_model.get_layer('层的名字')

## TFRecords操作

In [26]:
def _bytes_feature(value):
    if isinstance(value,type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [27]:
print(_bytes_feature(b'hello world'))

bytes_list {
  value: "hello world"
}



In [30]:
print(_float_feature(0.11))

float_list {
  value: 0.10999999940395355
}



### tfrecore制作方法

In [ ]:
def serialize_example(feature0,feature1,feature2,feature3):
    feature = {
        'feature0':_int64_feature(feature0),
        'feature1':_int64_feature(feature1),
        'feature2':_bytes_feature(feature2),
        'feature3':_float_feature(feature3),
    }
    example_proto = tf.trian.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
with.tf.io.TFRecordWriter(fiename) as writer:
    for i in range(n):
        example = serialize_example(feature0[i],feature1[i],feature2[i],feature3[i])
        writer.write(example)
        
raw_dataset = tf.data.TFRecordDataset(filename)